In [1]:
import pandas as pd
import numpy as np


In [18]:
csv_data = pd.read_csv("../datasets/audio_data.csv")

In [19]:
data = csv_data.to_numpy()

In [7]:
csv_data = csv_data.drop(index=0,axis=0)

In [11]:
temp = pd.read_csv("data.csv")

In [20]:
num_dirs = data[:,49]
deposit_amt = data[:,50]
max_spd = data[:,51]
max_frc=data[:,52]
dec_rate=data[:,53]
sense_dist=data[:,54]
slope_step=data[:,55]
disp_size=data[:,56]

In [21]:
num_dirs = np.delete(num_dirs, (0), axis=0)
deposit_amt = np.delete(deposit_amt, (0), axis=0)
max_spd = np.delete(max_spd, (0), axis=0)
max_frc = np.delete(max_frc, (0), axis=0)
dec_rate = np.delete(dec_rate, (0), axis=0)
sense_dist = np.delete(sense_dist, (0), axis=0)
slope_step = np.delete(slope_step, (0), axis=0)
disp_size = np.delete(disp_size, (0), axis=0)

In [26]:
num_dirs[0]

2.0

In [22]:
x = data[:,:49]

In [11]:
csv_data.head()

,avg_onset,spec_centroid,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,...,spec_spread,spec_flux,num_dirs,deposit_amt,max_spd,max_frc,dec_rate,sense_dist,slope_step,disp_size
1,1466.093018,1923.234009,0.0,0.333913,0.443973,0.336422,0.335821,0.283703,0.283886,0.273554,...,4423.216309,1.26222,2,8,6,6,4,3,1,1
2,1466.093018,1923.234009,0.0,0.333913,0.443973,0.336422,0.335821,0.283703,0.283886,0.273554,...,4423.216309,1.26222,2,8,6,6,4,3,1,1
3,1466.093018,1923.234009,0.0,0.333913,0.443973,0.336422,0.335821,0.283703,0.283886,0.273554,...,4423.216309,1.26222,2,8,6,6,4,3,1,1
4,1466.093018,1923.234009,0.0,0.333913,0.443973,0.336422,0.335821,0.283703,0.283886,0.273554,...,4423.216309,1.26222,2,8,6,6,4,3,1,1
5,1466.093018,1923.234009,0.0,0.333913,0.443973,0.336422,0.335821,0.283703,0.283886,0.273554,...,4423.216309,1.26222,2,8,6,6,4,3,1,1


In [23]:
x = np.delete(x, (0), axis=0)

In [24]:
from sklearn.preprocessing import MinMaxScaler


In [25]:
scaler = MinMaxScaler(feature_range=(0,1))
x_norm = scaler.fit_transform(x)

In [26]:
x.shape

(38046, 49)

In [27]:
num_dirs.shape

(38046,)

In [28]:
for i in range(len(num_dirs)):
    num_dirs[i] = num_dirs[i] -1

In [13]:
import collections
collections.Counter(num_dirs)

Counter({0.0: 1316,
         1.0: 1202,
         2.0: 3953,
         3.0: 2149,
         4.0: 1570,
         6.0: 1643,
         7.0: 3434,
         8.0: 4955,
         9.0: 1467,
         10.0: 1180,
         12.0: 15177})

In [29]:
from sklearn.utils import shuffle

In [30]:
x_norm, num_dirs, deposit_amt, max_spd, max_frc, dec_rate,sense_dist,slope_step,disp_size = shuffle(x_norm, num_dirs, deposit_amt, max_spd, max_frc, dec_rate,sense_dist,slope_step,disp_size)

In [31]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dense, LeakyReLU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.metrics import categorical_crossentropy

In [32]:
from tensorflow.python.keras.callbacks import TensorBoard

In [33]:
NAME ="num_dir"
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [35]:
train_x = x_norm
train_num_dirs = num_dirs

# NUM DIRECTIONS MODEL

In [34]:
num_dir_model = Sequential([
    Dense(units=128, input_shape=(49,), activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=512,activation='relu'),
    Dense(units=13, activation='softmax')
])
num_dir_model.compile(optimizer=Adam(lr=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [36]:
num_dir_model.fit(x=train_x,
                  y=train_num_dirs,
                  batch_size=10,
                  verbose=2, 
                  validation_split=.01,
                  epochs=20,
                  callbacks=[tensorboard])

Train on 37665 samples, validate on 381 samples
Epoch 1/20
 - 48s - loss: 1.4085 - acc: 0.5386 - val_loss: 1.1052 - val_acc: 0.6247
Epoch 2/20
 - 49s - loss: 0.9955 - acc: 0.6688 - val_loss: 0.8905 - val_acc: 0.6850
Epoch 3/20
 - 65s - loss: 0.8185 - acc: 0.7255 - val_loss: 0.7430 - val_acc: 0.7638
Epoch 4/20
 - 60s - loss: 0.6848 - acc: 0.7702 - val_loss: 0.6362 - val_acc: 0.7822
Epoch 5/20
 - 51s - loss: 0.5715 - acc: 0.8105 - val_loss: 0.5405 - val_acc: 0.8163
Epoch 6/20
 - 51s - loss: 0.4789 - acc: 0.8421 - val_loss: 0.4360 - val_acc: 0.8766
Epoch 7/20
 - 53s - loss: 0.4007 - acc: 0.8719 - val_loss: 0.3645 - val_acc: 0.8871
Epoch 8/20
 - 50s - loss: 0.3396 - acc: 0.8953 - val_loss: 0.3003 - val_acc: 0.9265
Epoch 9/20
 - 50s - loss: 0.2876 - acc: 0.9145 - val_loss: 0.2463 - val_acc: 0.9344
Epoch 10/20
 - 53s - loss: 0.2463 - acc: 0.9292 - val_loss: 0.2302 - val_acc: 0.9318
Epoch 11/20
 - 51s - loss: 0.2118 - acc: 0.9404 - val_loss: 0.2148 - val_acc: 0.9265
Epoch 12/20
 - 52s - loss:

In [37]:
num_dir_model.save('num_dir.h5')

# DEPOSIT AMOUNT MODEL

In [38]:
for i in range(len(deposit_amt)):
    deposit_amt[i] = deposit_amt[i] -1

In [39]:
train_deposit_amt = deposit_amt

In [40]:
NAME2 ="deposit_amt"
tensorboard2 = TensorBoard(log_dir="logs/{}".format(NAME2))

In [41]:
dep_amt_model = Sequential([
    Dense(units=128, input_shape=(49,), activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=512,activation='relu'),
    Dense(units=10, activation='softmax')
])
dep_amt_model.compile(optimizer=Adam(lr=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [42]:
dep_amt_model.fit(x=train_x,
                  y=train_deposit_amt,
                  batch_size=10,
                  validation_split=.1,
                  epochs=20,
                  callbacks=[tensorboard2],
                 verbose=2)

Train on 34241 samples, validate on 3805 samples
Epoch 1/20
 - 42s - loss: 1.4572 - acc: 0.4799 - val_loss: 1.1704 - val_acc: 0.5869
Epoch 2/20
 - 43s - loss: 1.0520 - acc: 0.6254 - val_loss: 0.9453 - val_acc: 0.6752
Epoch 3/20
 - 48s - loss: 0.8562 - acc: 0.7067 - val_loss: 0.7668 - val_acc: 0.7466
Epoch 4/20
 - 48s - loss: 0.7082 - acc: 0.7666 - val_loss: 0.6510 - val_acc: 0.7848
Epoch 5/20
 - 46s - loss: 0.5925 - acc: 0.8131 - val_loss: 0.5911 - val_acc: 0.7916
Epoch 6/20
 - 48s - loss: 0.5011 - acc: 0.8429 - val_loss: 0.4878 - val_acc: 0.8544
Epoch 7/20
 - 49s - loss: 0.4301 - acc: 0.8691 - val_loss: 0.4153 - val_acc: 0.8678
Epoch 8/20
 - 49s - loss: 0.3732 - acc: 0.8897 - val_loss: 0.3513 - val_acc: 0.8965
Epoch 9/20
 - 47s - loss: 0.3244 - acc: 0.9060 - val_loss: 0.3016 - val_acc: 0.9151
Epoch 10/20
 - 48s - loss: 0.2857 - acc: 0.9199 - val_loss: 0.2683 - val_acc: 0.9238
Epoch 11/20
 - 47s - loss: 0.2494 - acc: 0.9317 - val_loss: 0.2395 - val_acc: 0.9340
Epoch 12/20
 - 50s - loss

In [43]:
dep_amt_model.save('deposit_amt.h5')


# MAX SPEED MODEL

In [44]:
for i in range(len(max_spd)):
    max_spd[i] = max_spd[i] -1

In [45]:
train_max_spd = max_spd

In [46]:
NAME3 ="max_speed"
tensorboard3 = TensorBoard(log_dir="logs/{}".format(NAME3))

In [47]:
max_spd_model = Sequential([
    Dense(units=128, input_shape=(49,), activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=512,activation='relu'),
    Dense(units=10, activation='softmax')
])
max_spd_model.compile(optimizer=Adam(lr=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [48]:
max_spd_model.fit(x=train_x,
                  y=train_max_spd,
                  batch_size=10,
                  validation_split=.01,
                  epochs=20,
                  callbacks=[tensorboard3])

Train on 37665 samples, validate on 381 samples
Epoch 1/20
37665/37665 [==============================]37665/37665 [==============================] - 62s 2ms/step - loss: 1.3360 - acc: 0.4951 - val_loss: 1.0098 - val_acc: 0.6404

Epoch 2/20
37665/37665 [==============================]37665/37665 [==============================] - 73s 2ms/step - loss: 0.9463 - acc: 0.6568 - val_loss: 0.8176 - val_acc: 0.6824

Epoch 3/20
37665/37665 [==============================]37665/37665 [==============================] - 71s 2ms/step - loss: 0.7685 - acc: 0.7286 - val_loss: 0.7246 - val_acc: 0.7375

Epoch 4/20
37665/37665 [==============================]37665/37665 [==============================] - 75s 2ms/step - loss: 0.6398 - acc: 0.7774 - val_loss: 0.5809 - val_acc: 0.8005

Epoch 5/20
37665/37665 [==============================]37665/37665 [==============================] - 71s 2ms/step - loss: 0.5428 - acc: 0.8153 - val_loss: 0.4689 - val_acc: 0.8530

Epoch 6/20
37665/37665 [==================

In [49]:
max_spd_model.save('max_speed.h5')

# MAX FORCE MODEL

In [50]:
for i in range(len(max_frc)):
    max_frc[i] = max_frc[i] -1
    
train_max_frc = max_frc

NAME4 ="max_force"
tensorboard4 = TensorBoard(log_dir="logs/{}".format(NAME4))

max_frc_model = Sequential([
    Dense(units=128, input_shape=(49,), activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=512,activation='relu'),
    Dense(units=13, activation='softmax')
])

max_frc_model.compile(optimizer=Adam(lr=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [51]:
max_frc_model.fit(x=train_x,
                  y=train_max_frc,
                  batch_size=10,
                 
                  validation_split=.1,
                  epochs=20,
                  callbacks=[tensorboard4])

Train on 34241 samples, validate on 3805 samples
Epoch 1/20
34241/34241 [==============================]34241/34241 [==============================] - 55s 2ms/step - loss: 1.4965 - acc: 0.4655 - val_loss: 1.2186 - val_acc: 0.5530

Epoch 2/20
34241/34241 [==============================]34241/34241 [==============================] - 57s 2ms/step - loss: 1.1042 - acc: 0.6042 - val_loss: 0.9841 - val_acc: 0.6565

Epoch 3/20
34241/34241 [==============================]34241/34241 [==============================] - 66s 2ms/step - loss: 0.9249 - acc: 0.6764 - val_loss: 0.8660 - val_acc: 0.6922

Epoch 4/20
34241/34241 [==============================]34241/34241 [==============================] - 68s 2ms/step - loss: 0.7870 - acc: 0.7341 - val_loss: 0.7124 - val_acc: 0.7653

Epoch 5/20
34241/34241 [==============================]34241/34241 [==============================] - 68s 2ms/step - loss: 0.6760 - acc: 0.7759 - val_loss: 0.6754 - val_acc: 0.7653

Epoch 6/20
34241/34241 [=================

In [52]:
max_frc_model.save('max_force.h5')

# DECAY RATE MODEL

In [53]:
for i in range(len(dec_rate)):
    dec_rate[i] = dec_rate[i] -1
    
train_dec_rate = dec_rate

NAME5 ="decay_rate"
tensorboard5 = TensorBoard(log_dir="logs/{}".format(NAME5))

dec_rate_model = Sequential([
    Dense(units=128, input_shape=(49,), activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=512,activation='relu'),
    Dense(units=4, activation='softmax')
])

dec_rate_model.compile(optimizer=Adam(lr=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
dec_rate_model.fit(x=train_x,
                  y=train_dec_rate,
                  batch_size=10,
                  validation_split=.1,
                  epochs=20,
                  callbacks=[tensorboard5])

Train on 34241 samples, validate on 3805 samples
Epoch 1/20
34241/34241 [==============================]34241/34241 [==============================] - 56s 2ms/step - loss: 0.5563 - acc: 0.7818 - val_loss: 0.4527 - val_acc: 0.8223

Epoch 2/20
34241/34241 [==============================]34241/34241 [==============================] - 59s 2ms/step - loss: 0.4001 - acc: 0.8387 - val_loss: 0.3480 - val_acc: 0.8620

Epoch 3/20
34241/34241 [==============================]34241/34241 [==============================] - 60s 2ms/step - loss: 0.3161 - acc: 0.8754 - val_loss: 0.2733 - val_acc: 0.9004

Epoch 4/20
34241/34241 [==============================]34241/34241 [==============================] - 61s 2ms/step - loss: 0.2480 - acc: 0.9060 - val_loss: 0.2194 - val_acc: 0.9185

Epoch 5/20
34241/34241 [==============================]34241/34241 [==============================] - 61s 2ms/step - loss: 0.1954 - acc: 0.9263 - val_loss: 0.1561 - val_acc: 0.9398

Epoch 6/20
34241/34241 [=================

In [55]:
dec_rate_model.save('decay_rate.h5')

# SENSE DIST MODEL

In [56]:
for i in range(len(sense_dist)):
    sense_dist[i] = sense_dist[i] -1
    
train_sense_dist = sense_dist

NAME6 ="sense_dist"
tensorboard6 = TensorBoard(log_dir="logs/{}".format(NAME6))

sense_dist_model = Sequential([
    Dense(units=128, input_shape=(49,), activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=512,activation='relu'),
    Dense(units=12, activation='softmax')
])

sense_dist_model.compile(optimizer=Adam(lr=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [57]:
sense_dist_model.fit(x=train_x,
                  y=train_sense_dist,
                  batch_size=10,
                  validation_split=.1,
                  epochs=20,
                  callbacks=[tensorboard6])

Train on 34241 samples, validate on 3805 samples
Epoch 1/20
34241/34241 [==============================]34241/34241 [==============================] - 54s 2ms/step - loss: 1.4395 - acc: 0.4989 - val_loss: 1.1815 - val_acc: 0.5690

Epoch 2/20
34241/34241 [==============================]34241/34241 [==============================] - 55s 2ms/step - loss: 1.0504 - acc: 0.6311 - val_loss: 0.9776 - val_acc: 0.6625

Epoch 3/20
34241/34241 [==============================]34241/34241 [==============================] - 60s 2ms/step - loss: 0.8945 - acc: 0.6931 - val_loss: 0.8478 - val_acc: 0.7064

Epoch 4/20
34241/34241 [==============================]34241/34241 [==============================] - 58s 2ms/step - loss: 0.7792 - acc: 0.7349 - val_loss: 0.7475 - val_acc: 0.7390

Epoch 5/20
34241/34241 [==============================]34241/34241 [==============================] - 61s 2ms/step - loss: 0.6812 - acc: 0.7738 - val_loss: 0.6558 - val_acc: 0.7727

Epoch 6/20
34241/34241 [=================

In [58]:
sense_dist_model.save('sense_dist.h5')

# SLOPE STEP MODEL

In [59]:
for i in range(len(slope_step)):
    slope_step[i] = slope_step[i] -1
    
train_slope_step = slope_step

NAME7 ="slope_step"
tensorboard7 = TensorBoard(log_dir="logs/{}".format(NAME7))

slope_step_model = Sequential([
    Dense(units=128, input_shape=(49,), activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=512,activation='relu'),
    Dense(units=6, activation='softmax')
])

slope_step_model.compile(optimizer=Adam(lr=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [60]:
slope_step_model.fit(x=train_x,
                  y=train_slope_step,
                  batch_size=10,
                  validation_split=.1,
                  epochs=20,
                  callbacks=[tensorboard7])

Train on 34241 samples, validate on 3805 samples
Epoch 1/20
34241/34241 [==============================]34241/34241 [==============================] - 53s 2ms/step - loss: 0.3983 - acc: 0.8754 - val_loss: 0.2704 - val_acc: 0.9093

Epoch 2/20
34241/34241 [==============================]34241/34241 [==============================] - 59s 2ms/step - loss: 0.2203 - acc: 0.9241 - val_loss: 0.1747 - val_acc: 0.9369

Epoch 3/20
34241/34241 [==============================]34241/34241 [==============================] - 66s 2ms/step - loss: 0.1526 - acc: 0.9478 - val_loss: 0.1359 - val_acc: 0.9606

Epoch 4/20
34241/34241 [==============================]34241/34241 [==============================] - 67s 2ms/step - loss: 0.1110 - acc: 0.9613 - val_loss: 0.1065 - val_acc: 0.9698

Epoch 5/20
34241/34241 [==============================]34241/34241 [==============================] - 70s 2ms/step - loss: 0.0833 - acc: 0.9718 - val_loss: 0.0705 - val_acc: 0.9777

Epoch 6/20
34241/34241 [=================

In [61]:
slope_step_model.save('slope_step.h5')

# DISPLACE SIZE MODEL

In [62]:
for i in range(len(disp_size)):
    disp_size[i] = disp_size[i] -1
    
train_disp_size =disp_size

NAME8 ="displace_size"
tensorboard8 = TensorBoard(log_dir="logs/{}".format(NAME8))

disp_size_model = Sequential([
    Dense(units=128, input_shape=(49,), activation='relu'),
    Dense(units=256,activation='relu'),
    Dense(units=512,activation='relu'),
    Dense(units=8, activation='softmax')
])

disp_size_model.compile(optimizer=Adam(lr=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [63]:
disp_size_model.fit(x=train_x,
                  y=train_disp_size,
                  batch_size=10,
                  validation_split=.1,
                  epochs=20,
                  callbacks=[tensorboard8])

Train on 34241 samples, validate on 3805 samples
Epoch 1/20
34241/34241 [==============================]34241/34241 [==============================] - 56s 2ms/step - loss: 0.3850 - acc: 0.8812 - val_loss: 0.2309 - val_acc: 0.9191

Epoch 2/20
34241/34241 [==============================]34241/34241 [==============================] - 62s 2ms/step - loss: 0.2028 - acc: 0.9291 - val_loss: 0.1578 - val_acc: 0.9488

Epoch 3/20
34241/34241 [==============================]34241/34241 [==============================] - 69s 2ms/step - loss: 0.1454 - acc: 0.9491 - val_loss: 0.1282 - val_acc: 0.9601

Epoch 4/20
34241/34241 [==============================]34241/34241 [==============================] - 69s 2ms/step - loss: 0.1056 - acc: 0.9646 - val_loss: 0.0971 - val_acc: 0.9635

Epoch 5/20
34241/34241 [==============================]34241/34241 [==============================] - 71s 2ms/step - loss: 0.0797 - acc: 0.9749 - val_loss: 0.0742 - val_acc: 0.9798

Epoch 6/20
34241/34241 [=================

In [21]:
disp_size_model.save('displace_size.h5')